In [1]:
#recursos para trabar los df
import pandas as pd 
import numpy as np 
import ast 


In [2]:
import json

In [3]:
#recursos para normalizar el texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [4]:
rows = []
with open('australian_user_reviews.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
        df_games_reviews = pd.DataFrame(rows)

In [5]:
df_games = pd.read_json('output_steam_games.json',lines=True)

In [6]:
with open("australian_users_items.json", 'r', encoding= 'utf-8')as f:

    data = f.readlines()

records = [eval(line.strip())for line in data]

df_games_items = pd.DataFrame(records)


desenredo la columna reviews usando la funcion explode


In [7]:
df_games_reviews2 = df_games_reviews.explode('reviews')
df_games_reviews2 = df_games_reviews.explode('reviews')

funny = []
posted = []
last_edited = [] 
item_id = []
helpful = []
recommend = []
review = []

for index, row in df_games_reviews2.iterrows():
    elemento = row['reviews']
    
    if isinstance(elemento, dict):
        funny.append(elemento.get('funny', ''))
        posted.append(elemento.get('posted', ''))
        last_edited.append(elemento.get('last_edited', ''))
        item_id.append(elemento.get('item_id', ''))
        helpful.append(elemento.get('helpful', ''))
        recommend.append(elemento.get('recommend', ''))
        review.append(elemento.get('review', ''))
    else:
        funny.append('')
        posted.append('')
        last_edited.append('')
        item_id.append('')
        helpful.append('')
        recommend.append('')
        review.append('')

df_games_reviews2['funny'] = funny
df_games_reviews2['posted'] = posted
df_games_reviews2['last_edited'] = last_edited
df_games_reviews2['item_id'] = item_id
df_games_reviews2['helpful'] = helpful
df_games_reviews2['recommend'] = recommend
df_games_reviews2['review'] = review


In [8]:
#siendo que ya pase toda la info de reviews a otras columnas ahora la borro
df_games_reviews2.drop('reviews',axis=1, inplace=True)

In [9]:
#cambio todos los no ratings yet por np.nans y si no es esa frase sigue estando lo que habia
df_games_reviews2['helpful'] = np.where(df_games_reviews2['helpful']== 'No ratings yet',  np.nan, df_games_reviews2['helpful'])

In [10]:
# Supongamos que 'date_column' es el nombre de la columna con las fechas en formato de cadena
pattern = r'Posted (\w+ \d+, \d{4})'

# Filtrar y extraer las fechas completas
df_games_reviews2['posted'] = df_games_reviews2['posted'].str.extract(pattern)[0]

# Convertir las fechas completas en formato de fecha
df_games_reviews2['posted'] = pd.to_datetime(df_games_reviews2['posted'], format='%B %d, %Y')

# Ahora df['date_column'] contiene las fechas completas en formato de fecha

In [11]:
#columna inutil
df_games_reviews2.drop('last_edited',axis=1, inplace=True)

In [12]:
#otra columna inecesaria
df_games_reviews2.drop('funny',axis=1, inplace=True)

esto elimina donde todo es nulo, osea filas sin sentido alguno

In [13]:
df_games.dropna(how='all',inplace=True)

lo mismo que reviews pero en items

In [14]:
df_games_items2 = df_games_items.explode('items')

item_id = []
item_name = []
playtime_forever = [] 
playtime_2weeks = []

for index, row in df_games_items2.iterrows():
    elemento = row['items']
    
    if isinstance(elemento, dict):
        item_id.append(elemento.get('item_id', None))
        item_name.append(elemento.get('item_name', None))
        playtime_forever.append(elemento.get('playtime_forever', None))
        playtime_2weeks.append(elemento.get('playtime_2weeks', None))
    else:
        item_id.append(None)
        item_name.append(None)
        playtime_forever.append(None)
        playtime_2weeks.append(None)

df_games_items2['item_id '] = item_id 
df_games_items2['item_name'] = item_name
df_games_items2['playtime_forever'] = playtime_forever 
df_games_items2['playtime_2weeks '] = playtime_2weeks


una vez desarmada ya no es necesario la columna

In [15]:
#ya la desenrede asi que la borro
df_games_items2.drop('items',axis=1, inplace=True)

In [16]:
#una hermosa y trabajosa lista de emojis
emoji_to_text = {
    ':-)': 'Smile',
    ';-)': 'Smile with a wink',
    ':<})': 'User with moustache, smiling',
    ':-||': 'Mad',
    ':-((': 'Sad',
    ":'-(": "Crying:~Also crying",
    ':-))': 'Really happy:-DBig grin',
    ':-*': 'A kiss',
    ':-P~': 'A lick',
    ':-o': "Wow! or I'm surprised",
    ':-|': 'Grim:-PSticking out your tongue',
    ':-/': 'perplexed',
    '=:o': 'frightened',
    ':D': 'laugh',
    ':3': 'smile',
    'xD': 'laugh loud',
    'ô•': 'oh',
    ':c': 'addictive',
    ':)': 'happy',
    ':p': 'boring',
    '^^': 'good',
    'D=': 'pain',
    ':(': 'sad',
    'xd': 'happiness',
    '=P': 'crazy',
    '<3': 'heart',
    ';>': 'blink',
    '(Y)': 'thumbs up',
    '---': 'enjoyable',
    ',3': 'amazing',
    'xt': 'unwanted',
    '-': 'recommended',
    ':}': 'good',
    'c:': 'happy',
    ':v': 'want more',
    'x3': 'cute face',
    'XD': 'addictive',
    '(y)': 'thumbs up',
    ':P': 'bad',
    ';0': 'laugh',
    '=)': 'happy',
    'gg': 'great game',
    'GG': 'great game',
    ';D': 'great game',
    ':c': 'sad'
}

In [17]:
#cambio todos los emojis por las palabras que segun yo les corresponderia
def replace_emojis(text):
    for emoji, replacement in emoji_to_text.items():
        text = text.replace(emoji, replacement)
    return text

# Aplicar la función a la columna del DataFrame
df_games_reviews2['review'] = df_games_reviews2['review'].apply(replace_emojis)

In [18]:



# Definir una función para normalizar el texto
def normalize_text(text):
    # Tokenizar el texto en palabras
    words = word_tokenize(text)
    
    # Convertir todas las palabras a minúsculas
    words = [word.lower() for word in words]
    
    # Eliminar signos de puntuación
    words = [word for word in words if word not in string.punctuation]
    
    # Eliminar stopwords (palabras vacías)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Lematizar las palabras (convertirlas a su forma base)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Unir las palabras de nuevo en una cadena de texto
    normalized_text = ' '.join(words)
    
    return normalized_text

# Aplicar la función de normalización a la columna 'review'
df_games_reviews2['normalized_review'] = df_games_reviews2['review'].apply(normalize_text)



In [19]:
#analisis de sentimiento
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
sid = SentimentIntensityAnalyzer()

# Función para calcular la evaluación de sentimiento
def get_sentiment_scores(text):
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores

# Aplicar la función a la columna 'review' y crear una nueva columna 'sentiment_scores'
df_games_reviews2['sentiment_scores'] = df_games_reviews2['normalized_review'].apply(get_sentiment_scores)
# Aplicar la función a la columna 'review' y crear una nueva columna 'sentiment_scores'
df_games_reviews2['sentiment_scores'] = df_games_reviews2['normalized_review'].apply(get_sentiment_scores)

# Si deseas desglosar los puntajes de sentimiento en columnas individuales
# como 'compound', 'neg', 'neu' y 'pos', puedes hacer lo siguiente:
df_games_reviews2[['sentiment_compound', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos']] = df_games_reviews2['sentiment_scores'].apply(pd.Series)


In [20]:
df_games_reviews2.head(10)

,user_id,user_url,posted,item_id,helpful,recommend,review,normalized_review,sentiment_scores,sentiment_compound,sentiment_neg,sentiment_neu,sentiment_pos
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,NaN,True,Simple yet with great replayability. In my opi...,simple yet great replayability opinion `` zomb...,"{'neg': 0.098, 'neu': 0.594, 'pos': 0.308, 'co...",0.098,0.594,0.308,0.8481
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200,NaN,True,It's unique and worth a playthrough.,'s unique worth playthrough,"{'neg': 0.0, 'neu': 0.612, 'pos': 0.388, 'comp...",0.000,0.612,0.388,0.2263
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110,NaN,True,Great atmosphere. The gunplay can be a bit chu...,great atmosphere gunplay bit chunky time end d...,"{'neg': 0.0, 'neu': 0.547, 'pos': 0.453, 'comp...",0.000,0.547,0.453,0.8910
1,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,know think see title `` barbie dreamhouse part...,"{'neg': 0.162, 'neu': 0.601, 'pos': 0.237, 'co...",0.162,0.601,0.237,0.5532
1,js41637,http://steamcommunity.com/id/js41637,2013-09-08,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,simple 's actually simple truck driving simula...,"{'neg': 0.061, 'neu': 0.619, 'pos': 0.32, 'com...",0.061,0.619,0.320,0.9550
1,js41637,http://steamcommunity.com/id/js41637,2013-11-29,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...,fun little game play bored time passer gud rec...,"{'neg': 0.108, 'neu': 0.352, 'pos': 0.541, 'co...",0.108,0.352,0.541,0.7495
2,evcentric,http://steamcommunity.com/id/evcentric,NaT,248820,NaN,True,A suitably punishing roguelike platformer. Wi...,suitably punishing roguelike platformer winnin...,"{'neg': 0.286, 'neu': 0.349, 'pos': 0.366, 'co...",0.286,0.349,0.366,0.3658
2,evcentric,http://steamcommunity.com/id/evcentric,2015-12-04,370360,NaN,True,"""Run for fun? What the hell kind of fun is that?""",`` run fun hell kind fun '',"{'neg': 0.261, 'neu': 0.17, 'pos': 0.568, 'com...",0.261,0.170,0.568,0.6597
2,evcentric,http://steamcommunity.com/id/evcentric,2014-11-03,237930,NaN,True,"Elegant integration of gameplay, story, world ...",elegant integration gameplay story world devel...,"{'neg': 0.0, 'neu': 0.659, 'pos': 0.341, 'comp...",0.000,0.659,0.341,0.4767
2,evcentric,http://steamcommunity.com/id/evcentric,2014-10-15,263360,NaN,True,"Random drops and random quests, with stat poin...",random drop random quest stat point animation ...,"{'neg': 0.16, 'neu': 0.84, 'pos': 0.0, 'compou...",0.160,0.840,0.000,-0.2732


In [48]:
df_games_reviews2['recommend'].astype
df_games_reviews2['recommend'] = df_games_reviews2['recommend'].replace({True: 1, False: 0})
#cambio la recomendacion por 1 si es recomendado o 0 si no

In [23]:
#normalizo los precios ya que habia muchas forams de decir que eran gratis, asi que si tienen texto como free to play valen 0
def convertir_a_cero(valor):
    if isinstance(valor, str):
        return 0
    return valor

# Aplicar la función a la columna 'Columna' del DataFrame
df_games['price']= df_games['price'].apply(convertir_a_cero)

In [24]:
#los convierto en flotantes para poder hacer cuentas 
df_games['price'].astype(float)

88310     4.99
88311     0.00
88312     0.00
88313     0.99
88314     2.99
          ... 
120440    1.99
120441    4.99
120442    1.99
120443    4.99
120444    4.99
Name: price, Length: 32135, dtype: float64

In [27]:
df_games.head(10)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,game
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,Lost Summoner Kitty
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL,Ironbound
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com,Real Pool 3D - Poolians
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域,弹炸人2222
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None,Log Challenge
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0.0,772540.0,Trickjump Games Ltd,Battle Royale Trainer
88316,None,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,SNOW - All Access Basic Pass,http://store.steampowered.com/app/774276/SNOW_...,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774276/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",9.99,0.0,774276.0,Poppermost Productions,SNOW - All Access Basic Pass
88317,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Pro Pass,SNOW - All Access Pro Pass,http://store.steampowered.com/app/774277/SNOW_...,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774277/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",18.99,0.0,774277.0,Poppermost Productions,SNOW - All Access Pro Pass
88318,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Legend Pass,SNOW - All Access Legend Pass,http://store.steampowered.com/app/774278/SNOW_...,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774278/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",29.99,0.0,774278.0,Poppermost Productions,SNOW - All Access Legend Pass
88319,RewindApp,"[Casual, Indie, Racing, Simulation]",Race,Race,http://store.steampowered.com/app/768800/Race/,2018-01-04,"[Indie, Casual, Simulation, Racing]",http://steamcommunity.com/app/768800/reviews/?...,"[Single-player, Multi-player, Partial Controll...",NaN,0.0,768800.0,RewindApp,Race


In [25]:
#por comodidad y no perder data uni estos 2 ya que decian lo mismo y si falta uno el otro lo reeemplaza
df_games['game'] = df_games['app_name'].fillna(df_games['title'])

In [28]:
#los borro porque ya no los necesito
columns_to_drop = ['app_name', 'title']

# Utiliza la función drop para eliminar las columnas
df_games.drop(columns=columns_to_drop, inplace=True)

In [29]:
df_games.head(10)

,publisher,genres,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,game
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,Lost Summoner Kitty
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL,Ironbound
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com,Real Pool 3D - Poolians
88313,彼岸领域,"[Action, Adventure, Casual]",http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域,弹炸人2222
88314,None,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None,Log Challenge
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0.0,772540.0,Trickjump Games Ltd,Battle Royale Trainer
88316,None,"[Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/774276/SNOW_...,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774276/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",9.99,0.0,774276.0,Poppermost Productions,SNOW - All Access Basic Pass
88317,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/774277/SNOW_...,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774277/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",18.99,0.0,774277.0,Poppermost Productions,SNOW - All Access Pro Pass
88318,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/774278/SNOW_...,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774278/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",29.99,0.0,774278.0,Poppermost Productions,SNOW - All Access Legend Pass
88319,RewindApp,"[Casual, Indie, Racing, Simulation]",http://store.steampowered.com/app/768800/Race/,2018-01-04,"[Indie, Casual, Simulation, Racing]",http://steamcommunity.com/app/768800/reviews/?...,"[Single-player, Multi-player, Partial Controll...",NaN,0.0,768800.0,RewindApp,Race


In [30]:
# Crear una copia del DataFrame original
df_games_copy = df_games.copy()

# Asegurarse de que todos los valores en la columna "genres" sean listas
df_games_copy['genres'] = df_games_copy['genres'].apply(lambda x: [x] if isinstance(x, str) else x)

# Crear un conjunto de géneros únicos
genre_set = set()
for genres_list in df_games_copy['genres']:
    if genres_list is not None:
        genre_set.update(genres_list)

# Iterar sobre los géneros únicos y crear columnas
for genre in genre_set:
    df_games_copy[genre] = df_games_copy['genres'].apply(lambda x: 1 if (x is not None) and (genre in x) else 0)

# Combinar el DataFrame original con las columnas codificadas
df_combined = df_games_copy.drop(columns=['genres'])

# Imprimir el DataFrame resultante
print(df_combined)

               publisher                                                url   
88310          Kotoshiro  http://store.steampowered.com/app/761140/Lost_...  \
88311   Making Fun, Inc.  http://store.steampowered.com/app/643980/Ironb...   
88312       Poolians.com  http://store.steampowered.com/app/670290/Real_...   
88313               彼岸领域     http://store.steampowered.com/app/767400/2222/   
88314               None  http://store.steampowered.com/app/773570/Log_C...   
...                  ...                                                ...   
120440   Ghost_RUS Games  http://store.steampowered.com/app/773640/Colon...   
120441            Sacada  http://store.steampowered.com/app/733530/LOGis...   
120442      Laush Studio  http://store.steampowered.com/app/610660/Russi...   
120443          SIXNAILS  http://store.steampowered.com/app/658870/EXIT_...   
120444              None  http://store.steampowered.com/app/681550/Maze_...   

       release_date                                

ES como un .lower pero mas cool 


In [31]:
column_mapping = {
    'Web Publishing': 'web_publishing',
    'Audio Production': 'audio_production',
    'Strategy': 'strategy',
    'Adventure': 'adventure',
    'Photo Editing': 'photo_editing',
    'RPG': 'rpg',
    'Action': 'action',
    'Utilities': 'utilities',
    'Accounting': 'accounting',
    'Free to Play': 'free_to_play',
    'Massively Multiplayer': 'massively_multiplayer',
    'Education': 'education',
    'Software Training': 'software_training',
    'Animation &amp; Modeling': 'animation_modeling',
    'Racing': 'racing',
    'Casual': 'casual',
    'Design &amp; Illustration': 'design_illustration',
    'Early Access': 'early_access',
    'Simulation': 'simulation',
    'Sports': 'sports',
    'Indie': 'indie',
    'Video Production': 'video_production'
}

# Renombrar las columnas en el DataFrame
df_combined.columns = df_combined.columns.str.lower()

In [32]:
#elimina los juego que no tienen nombre
df_games_items2 = df_games_items2.dropna(subset=['item_name'])

In [55]:
df_games_reviews2.head(1)

,user_id,user_url,posted,item_id,helpful,recommend,normalized_review,sentiment_compound,sentiment_neg,sentiment_neu,sentiment_pos
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,NaN,1,simple yet great replayability opinion `` zomb...,0.098,0.594,0.308,0.8481


In [54]:
df_games_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             59333 non-null  object        
 1   user_url            59333 non-null  object        
 2   posted              49186 non-null  datetime64[ns]
 3   item_id             59333 non-null  object        
 4   helpful             29165 non-null  object        
 5   recommend           59333 non-null  object        
 6   normalized_review   59333 non-null  object        
 7   sentiment_compound  59333 non-null  float64       
 8   sentiment_neg       59333 non-null  float64       
 9   sentiment_neu       59333 non-null  float64       
 10  sentiment_pos       59333 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(6)
memory usage: 5.4+ MB


In [51]:
# Realiza una combinación de datos entre df_combined y df_games_items2 en la columna 'game'
combined_games = df_combined.merge(df_games_items2[['item_name', 'playtime_forever']], left_on='game', right_on='item_name', how='inner')

In [46]:
columnas_desde_la_decima = combined_games.columns[9:].tolist()

In [47]:
columnas_desde_la_decima

['developer',
 'game',
 'free to play',
 'racing',
 'web publishing',
 'action',
 'education',
 'indie',
 'massively multiplayer',
 'accounting',
 'early access',
 'photo editing',
 'adventure',
 'simulation',
 'audio production',
 'rpg',
 'animation &amp; modeling',
 'casual',
 'utilities',
 'video production',
 'software training',
 'sports',
 'design &amp; illustration',
 'strategy',
 'item_name',
 'playtime_forever',
 'total_playtime']

In [52]:


# 1. Agrupa y suma las horas totales de juego por categoría
category_columns = ['free to play',
 'racing',
 'web publishing',
 'action',
 'education',
 'indie',
 'massively multiplayer',
 'accounting',
 'early access',
 'photo editing',
 'adventure',
 'simulation',
 'audio production',
 'rpg',
 'animation &amp; modeling',
 'casual',
 'utilities',
 'video production',
 'software training',
 'sports',
 'design &amp; illustration',
 'strategy',
 'item_name',
 'playtime_forever',
 ]
combined_games['total_playtime'] = combined_games['playtime_forever']
category_playtime = combined_games.groupby('game')[category_columns + ['total_playtime']].sum().reset_index()

# 2. Ordena el resultado en función de las horas totales de juego
category_playtime = category_playtime.sort_values(by='total_playtime', ascending=False)

In [53]:
category_playtime

,game,free to play,racing,web publishing,action,education,indie,massively multiplayer,accounting,early access,...,casual,utilities,video production,software training,sports,design &amp; illustration,strategy,item_name,playtime_forever,total_playtime
1465,Counter-Strike: Global Offensive,0,0,0,43776,0,0,0,0,0,...,0,0,0,0,0,0,0,Counter-Strike: Global OffensiveCounter-Strike...,785184267.0,785184267.0
2730,Garry's Mod,0,0,0,0,0,43301,0,0,0,...,0,0,0,0,0,0,0,Garry's ModGarry's ModGarry's ModGarry's ModGa...,448366616.0,448366616.0
6490,Terraria,0,0,0,29239,0,29239,0,0,0,...,0,0,0,0,0,0,0,TerrariaTerrariaTerrariaTerrariaTerrariaTerrar...,154974541.0,154974541.0
7434,Warframe,25807,0,0,25807,0,0,0,0,0,...,0,0,0,0,0,0,0,WarframeWarframeWarframeWarframeWarframeWarfra...,124027703.0,124027703.0
1466,Counter-Strike: Source,0,0,0,24220,0,0,0,0,0,...,0,0,0,0,0,0,0,Counter-Strike: SourceCounter-Strike: SourceCo...,112612047.0,112612047.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,Animated Puzzles,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,Animated Puzzles,0.0,0.0
733,BattleQuiz,0,0,0,0,0,2,0,0,0,...,2,0,0,0,0,0,0,BattleQuizBattleQuiz,0.0,0.0
1210,Champions Of Chaos 2,0,0,0,0,0,3,0,0,0,...,3,0,0,0,0,0,3,Champions Of Chaos 2Champions Of Chaos 2Champi...,0.0,0.0
6121,Star Rogue,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,Star Rogue,0.0,0.0


In [ ]:
df_it=df_games_items2.copy()
columnas=['item_name','playtime_forever']
df_it=df_it[columnas]


In [ ]:
df_it2=df_games_items2.copy()

In [ ]:
columnas3=['item_name','playtime_forever',]

In [ ]:
df1 = df_combined.merge(df_it, left_on='game', right_on='item_name', how='inner')


In [ ]:
lista_de_columnas = df1.columns.tolist()


In [ ]:
['web_publishing', 'audio_production', 'photo_editing', 'free_to_play', 'massively_multiplayer', 'software_training', 'animation_modeling', 'design_illustration', 'video_production']

In [ ]:
columnas2=[
 'release_date',
 'adventure',
 'software training',
 'photo editing',
 'indie',
 'education',
 'action',
 'animation &amp; modeling',
 'early access',
 'free to play',
 'audio production',
 'massively multiplayer',
 'design &amp; illustration',
 'utilities',
 'sports',
 'racing',
 'rpg',
 'accounting',
 'simulation',
 'casual',
 'strategy',
 'video production',
 'playtime_forever','id']

df1= df1[columnas2]

si guard0 los id es seguro borrar los duplicados

In [ ]:
df1.drop_duplicates(inplace=True)

In [ ]:
#ya no hacen falta
df1.drop(columns='id',axis=1,inplace=True)

In [ ]:
c1=['url','publisher','tags','reviews_url','specs','price','developer','id','game','user_id','steam_id','user_url','item_id ','item_name', 'playtime_2weeks ']
c2=['early_access','items_count']
df1=df1.drop(c2, axis=1)
df1=df1.drop(c1, axis=1)

vuelvo realesetime en tipo fecha

In [ ]:
df1['release_date'] = pd.to_datetime(df1['release_date'], format='%Y-%m-%d', errors='coerce')